# Final form
I would propose using a final form in which there is an explicit convolution and where A and C are defined to be positive.


In [1]:
import numpy as np
from scipy.special import erf
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, FloatSlider
from IPython.display import display

%matplotlib widget

In [2]:
%matplotlib widget

# Evaluate original form

In the email, the suggested form is  
$I(t) = g(t, \Gamma) \ast \{ 1 + A H(-t) - A\exp(-t/t_{mag})  - C(1-\exp(-t/t_{mag})) + B \sin…)$  \}

$g(t, \Gamma)$
is a Gaussian with width defined in terms of full-width at half-maximum $\Gamma$ 

* $A$ is the drop 
* $C$ is the amount of order that fails to recover  

We think this should be the same as 

In [9]:
t = np.linspace(-20, 100, 50000)

def exp(x):
    y = np.exp(x)
    y[y>1] = 1
    return y


def convolution_form(t, Gamma, A, C, t_mag):
    dt = np.mean(np.abs(np.diff(t)))
    x = np.arange(-Gamma*5, Gamma*5+dt, dt)
    t_extended = np.arange(t.min() - 5*Gamma, t.max() + 5*Gamma, dt) 
    kernel = np.exp(-(2*np.sqrt(np.log(2))*x/Gamma)**2)
    kernel = kernel / np.sum(kernel)
    y = (1 + A*np.heaviside(-t_extended, 1) - A*exp(-t_extended/t_mag)
         - C*(1-exp(-t_extended/t_mag))
        )
    I_extended = np.convolve(y, kernel, mode='same')
    return np.interp(t, t_extended, I_extended)


def plot_revised_form(Gamma=.15, A=0.5, C=0.2, t_mag=5):
    I = convolution_form(t, Gamma=Gamma, A=A, C=C, t_mag=t_mag)
    art.set_ydata(I)

fig, ax = plt.subplots()
I = convolution_form(t, Gamma=.01, A=-.8, C=.5, t_mag=5)
art,  = ax.plot(t, I)
ax.set_xlabel('t')
ax.set_ylabel('I')
ax.set_ylim((-4, 10))

interact(plot_revised_form,
         Gamma=FloatSlider(value=0.15, min=0.00001, max=10, step=0.0001),
         A=FloatSlider(value=.9, min=-1, max=1),
         C=FloatSlider(value=.2, min=-1, max=1),
         t_mag=FloatSlider(value=.5, min=0.0001, max=5, step=0.0001),
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.15, description='Gamma', max=10.0, min=1e-05, step=0.0001), FloatSli…

<function __main__.plot_revised_form(Gamma=0.15, A=0.5, C=0.2, t_mag=5)>